<a href="https://colab.research.google.com/github/DonErnesto/masterclassSFI_2021/blob/main/notebooks/BitcoinSupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anti-Money Laundering with Supervised Classification

**Introduction**


The purpose of this Jupyter notebook is to make you familiar with several common supervised ML models, and to guide you through some essential steps when developing an ML model: hyperparameter tuning, model comparison and selection. 

Note that there are two types of cells in this notebook: **markdown cells** (that contain text, like this one), and **code cells** (that execute some code, like the next cell). 

By clicking the Play button on a cell, we execute a code cell. Lines that start with a "#" are comments, and not executed. Furthermore, note that correct **indentation** (use 4 spaces, always after a line that ends with a colon `:`) is mandatory.  


The data we will be using was taken from Kaggle: https://www.kaggle.com/ellipticco/elliptic-data-set 
and describes blockchain transactions, some of which are flagged as "illicit" (i.e., relating to illegal activity), others as "licit" or "unknown" (unknown being the majority, with about 80%). We got rid of the unknown labels for simplicity. The authors give as examples of illicit categories: "scams, malware, terrorist organizations, ransomware, Ponzi schemes, etc."

We start by downloading the data we will be training on, which has already been splitted into "X" (features) and "y" (labels).


In [ ]:
## Data import from Github
import os
force_download = False
if force_download or not os.path.exists('X_train_supervised.csv.zip'): # then probably nothing was downloaded yet
    !curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/ml_utils.py
    !curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/X_train_supervised.csv.zip
    !curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/y_train_supervised.csv.zip
    !curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/X_test_supervised.csv.zip
    !curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/y_test_supervised.csv.zip
    

We will be using pandas for data handling, and scikit-learn (sklearn) for various helper functions, and for its supervised machine learning algorithms. 

In [ ]:
# Package imports
import warnings
warnings.filterwarnings("ignore")

import pandas as pd # data I/O and manipulation
import numpy as np # numeric operations
import matplotlib.pyplot as plt # basic plotting

from sklearn.metrics import roc_auc_score, plot_roc_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from ml_utils import grouped_boxplot_gridsearch, plot_conditional_distribution # our own helper functions


Next, we will load the data in a so-called DataFrame (a pandas object), and inspect it by plotting the N-top rows

In [ ]:
X_train = pd.read_csv('X_train_supervised.csv.zip')
X_test = pd.read_csv('X_test_supervised.csv.zip')
y_train = pd.read_csv('y_train_supervised.csv.zip')['class']

In [ ]:
# .head() returns the first n (per default 5) rows of a DataFrame
X_train.head() 

In [ ]:
# We note that the feature extraction has been done for us, and that all
# data has already been properly prepared (all values are numerical).
# We therefore only remove unwanted features txId and Time step
cols_to_drop = ['txId', 'Time step']
X_train = X_train.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

**Further documentation on this dataset:**

From the Kaggle-website ( https://www.kaggle.com/ellipticco/elliptic-data-set ): "There are 166 features associated with each node. Due to intellectual property issues, we cannot provide an exact description of all the features in the dataset. There is a time step associated to each node, representing a measure of the time when a transaction was broadcasted to the Bitcoin network. ...

The first 94 features represent local information about the transaction – including the time step described above, number of inputs/outputs, transaction fee, output volume and aggregated figures such as average BTC received (spent) by the inputs/outputs and average number of incoming (outgoing) transactions associated with the inputs/outputs. The remaining 72 features are aggregated features, obtained using transaction information one-hop backward/forward from the center node - giving the maximum, minimum, standard deviation and correlation coefficients of the neighbour transactions for the same information data (number of inputs/outputs, transaction fee, etc.)."

Finally, and perhaps most relevant, the labels have been generated by a "heuristics-based reasoning process", as can be read in the article by Weber et al. (https://arxiv.org/pdf/1908.02591.pdf):

"For example, a higher number of inputs and the reuse of the same address is commonly associated with higher address-clustering, which results in a degrade of anonymity for the entity signing the transaction. On the other hand, consolidating funds controlled by multiple addresses in one single transaction provides benefits in terms of transaction costs (fee). It follows that entities eschewing anonymity-preserving measures for large volumes of user requests are likely to be licit (e.g. exchanges). In contrast, illicit activity may tend to favor transactions with a lower number of inputs to reduce the impact of de-anonymizing address-clustering techniques.". 

This essentially means that the labels were generated by hand-crafted rules. It is thus possible, that these rules may be reconstructed by the model through proxy-features, and that a higher-than-realistic score is achieved. 



In [ ]:
print('Shape of training data:', X_train.shape, '\n')
print('Label fractions:')
print(y_train.value_counts(normalize=True))
print(f'\nTest data size (fraction of total): {len(X_test)/(len(X_train) + len(X_test)):.2%} ')

There are 33.4k data points, of which 11% is a positive (which is quite a large fraction in a financial crime context). 28% of the data has been set aside for testing. 

We will not touch the test data until the very end. 

## Case-study: Decision Tree classifier

In [ ]:
# First we import the classes we want to use 
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
# Putting a question mark before (or after) a Python object shows you its documentation
?DecisionTreeClassifier

In [ ]:
# Then we instantiate the DecisionTreeClassifier and define the parameter space we want to explore
dtc = DecisionTreeClassifier(random_state=3) #Initialize with whatever parameters you want to

# we will vary the maximum depth of the tree, and the minimum required number of samples to make a split
param_grid = {'max_depth': [2, 5, 10, 20]} #Note the dictionary notation

In [ ]:
# We make use of the GridSearchCV estimator that does the scanning of the hyper-parameters for us,
# in a k-Fold cross-validation loop
grid_dtc = GridSearchCV(dtc, param_grid, cv=10, scoring='roc_auc', 
                        return_train_score=True) #NB: uses StratifiedKFold when cv=int

# Finally, we fit the GridSearchCV estimator to our training data, using the .fit() method
_ = grid_dtc.fit(X_train, y_train)

We use grouped_boxplot_gridsearch (a self-made helper function) to show how classifier performance is affected by the various hyperparameter settings. 

Note that the boxplots show:
- The median ("mid-point"), 
- a box spanning the first and third quartile, 
- and whiskers that extend to the median +/- 1.5 InterQuartile Range (IQR) or the lowest/highest point. Points beyond the median +/- 1.5 IQR are considered outliers and plotted explicitly

<div>
<img src="https://pro.arcgis.com/de/pro-app/2.7/help/analysis/geoprocessing/charts/GUID-0E2C3730-C535-40CD-8152-80D794A996A7-web.png" width="200">
</div>

In [ ]:
df = grouped_boxplot_gridsearch(grid_dtc, parameter_names=['max_depth'])
_ = plt.title('CV Test scores, Decision Tree Classifier')

Note how trees can always fit the training data perfectly (as long as there are no identical X's with different y's), given that they can grow large enough. This becomes apparent in the training scores (the scores on the training data itself, rather than on the validation set). 

In [ ]:
df = grouped_boxplot_gridsearch(grid_dtc, parameter_names=['max_depth' ], train_scores=True)
_ = plt.title('CV Train scores, Decision Tree Classifier')

In [ ]:
grid_dtc.estimator

Finally, let's visualize a tree. Let's make a simple tree, with max_depth=2

In [ ]:
demo_tree = DecisionTreeClassifier(max_depth=2, random_state=1)
demo_tree.fit(X_train, y_train)
fig, ax = plt.subplots(1,1, figsize=(12, 8))
_ = plot_tree(demo_tree, ax=ax)

Even with the simple Decision Tree Classifier, ROC-AUC close to 0.90 are feasible judging by the cross-validation scores. The variance is however quite high, so the final test score could disappoint us. 


# Break-out session

- Go the sub-section within `Model and hyper-parameter selection with cross-validation` belonging to your model and run the cells 
- For more accurate results you may increase `cv` to 5 (Neural Network) or 10 (others)
 
- Answer the following questions:
    - Do you see any signs of overfitting? (i.e. much better performance on the training than on the validation data)
    - What can you say about the bias and variance of the model?
    - What hyper-parameters help to regularize the model? In what direction?
    - Which set of hyper-parameter values would you choose, and which one is chosen by the GridSearchCV object?
- Go to the section `Evaluation on test data`, and evaluate the .best_estimator_ object within the fitted GridSearchCV object. This is the estimator with the highest mean cross-validation test metric (we have chosen ROC-AUC). 
    - What AUC do you get? Is it as expected based on your cross-validation results? 



## Model and hyper-parameter selection with cross-validation

### Naive Bayes classifier

The Naive Bayes classifier is a rather simple yet powerful classifier, that has been used successfully in for instance spam filters. Here we will use a classifier that assumes a Gaussian distribution of its features, the Gaussian Naive Bayes classifier.  

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
?GaussianNB

The Gaussian Naive Bayes classifier determines the mean and variance from the data. To avoid too small variances, the parameter `var_smoothing` boosts the variance of the features by epsilon, a fraction of the largest standard deviation of all features. 

In [ ]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('nb', GaussianNB())
])
param_grid = {'nb__var_smoothing': np.logspace(-12, -3, num=4)} #Note the dictionary notation
nb = GaussianNB()

In [ ]:
grid_nb = GridSearchCV(pipeline, param_grid, cv=10, scoring='roc_auc', return_train_score=True) #NB: uses StratifiedKFold when cv=int
_ = grid_nb.fit(X_train, y_train)

In [ ]:
df = grouped_boxplot_gridsearch(grid_nb, parameter_names=['nb__var_smoothing', ])
_ = plt.title('CV Test scores, Naive Bayes Classifier')

In [ ]:
# if desired, one can expect the detailed numerical results in the returned dataframe:
df

In [ ]:
df = grouped_boxplot_gridsearch(grid_nb, parameter_names=['nb__var_smoothing', ], train_scores=True)
_ = plt.title('CV Train scores, Naive Bayes Classifier')

### Logistic Regression

Logistic Regression is the classification-counterpart of Linear Least Squares for regression. Similar to linear regression, we can impose a penalty on larger coefficient values to prevent overfitting. This is called regularization. 

Too large a penalty (small C-value in the sklearn model) will lead to stable but sub-optimal performance (underfitting), too small a penalty may result in overfitting, especially when the number of features (columns) is high and the number of samples is low. When doing logistic regression, it is important to determine the optimal regularization strength in a cross-validation cycle. The default l2-penalty is chosen (penalizing the sum of squares of the coefficient values), this may be changed to a l1-penalty if desired. 

It is important that we scale the data before fitting the model when doing regularization. The correct way is to make scaling a part of a cross-validation pipeline, which is done using the Pipeline class of sklearn. 
The Pipeline object will behave just as a single classifier, having .fit() and .predict() methods. This means that  during cross-validation, also the parameters of the StandardScaler are fitted on the training split within the fold. 

**Question:** Why do you think the scaling is important when we regularize the model by penalizing the coefficients?

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
?LogisticRegression

In [ ]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(penalty='l2', solver='saga', n_jobs=-1, random_state=10))
])
# define the parameter grid, preceding the argument name with "lr__" when it applies to the LogisticRegression
param_grid = {'lr__C': np.logspace(-5, -1, num=5), 
              #'lr__penalty': ['l1', 'l2']
             } 
grid_lr = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', return_train_score=True)
grid_lr.fit(X_train, y_train)

In [ ]:
df = grouped_boxplot_gridsearch(grid_lr, parameter_names=['lr__C', ])
_ = plt.title('CV Test scores, Logistic Regression Classifier')

In [ ]:
df = grouped_boxplot_gridsearch(grid_lr, parameter_names=['lr__C', ], train_scores=True)
_ = plt.title('CV Train scores, Logistic Regression Classifier')

### Random Forest

Random Forest Classifiers typically perform quite well over a wide range of parameters. The main parameter to tune is the depth of the individual trees ('max_depth'), which determines the model complexity. Typically, when the number of trees ('n_estimators') is chosen large enough (say, 100 or more), Random Forest classifiers typically do not suffer from overfitting. This is because the classifier is an ensemble of many tree classifiers ("bagging"), which reduces the variance of the predictions. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
?RandomForestClassifier

In [ ]:
param_grid = {'max_depth': [2, 5, 10, 20], 'n_estimators':[10, 100]}
rfc = RandomForestClassifier(random_state=7) 

grid_rfc = GridSearchCV(rfc, param_grid, cv=5, scoring='roc_auc', return_train_score=True) 
_ = grid_rfc.fit(X_train, y_train)

In [ ]:
df = grouped_boxplot_gridsearch(grid_rfc, parameter_names=['max_depth', 'n_estimators'])
_ = plt.title('CV Test scores, Random Forest Classifier')

In [ ]:
df = grouped_boxplot_gridsearch(grid_rfc, parameter_names=['max_depth', 'n_estimators'], train_scores=True)
_ = plt.title('CV Train scores, Random Forest Classifier')

### Gradient boosted Trees

Gradient boosted trees share some similarities with Random Forests, in that they are an ensemble of trees. Whereas a Random Forest classifier consists of trees grown individually, Gradient Boosting generates trees that successively address misclassifications of the previous trees. Although scikit-learn does have a Gradient Boosting implementation it is advised to use LightGBM, a very performant implementation in terms of speed and accuracy. 

In [ ]:
# !pip install lightgbm
from sklearn.ensemble import GradientBoostingClassifier #scikit-learn implementation. Not advised
from lightgbm import LGBMClassifier #roughly 2 orders of magnitude times faster than scikit-learn's implementation

In [ ]:
clf_gb = LGBMClassifier()
#clf_gb = GradientBoostingClassifier()
param_grid = {
    'max_depth':[2, 5, 10], # sklearn and lightgbm implementation
    'num_iterations': [20, 50, 100], # lightgbm implementation
    }

grid_gb = GridSearchCV(estimator=clf_gb, param_grid=param_grid, cv=5, return_train_score=True, scoring='roc_auc')

In [ ]:
grid_gb.fit(X_train, y_train)

In [ ]:
df = grouped_boxplot_gridsearch(grid_gb, parameter_names=['max_depth', 'num_iterations' ])
_ = plt.title('CV Test scores, Gradient Boosted Tree Classifier')

In [ ]:
df = grouped_boxplot_gridsearch(grid_gb, parameter_names=['max_depth', 'num_iterations' ], train_scores=True)
_ = plt.title('CV Train scores, Gradient Boosted Tree Classifier')

### Feedforward Neural Network

A feedforward neural network consists of consecutive layers of densely connected neurons. Their weights and biases need to be trained using the training data. We make use of Tensorflow for speed of calculation, and keras as a wrapper around Tensorflow to make the construction of the neural network easier. We add dropout, a fast and efficient regularization technique.  Because dropout randomly disables neurons, more training rounds ("epochs") are needed and also a somewhat wider network architecture to achieve good results. 

Neural networks typically have a lot of parameters that can be tuned: the number of layers, the width of the layers, the activation functions to be used, the batch size, the optimizer, and in our case also the drop-out rate. 

We will take the default mini-batch size of 32, and scan for optimal dropout rate and network width. 

Note how many steps and design choices there are to be made to build a -rather simple- neural network architecture. 

In [ ]:
from tensorflow import keras
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
KerasClassifier._estimator_type = 'classifier' #monkey-patch the class for plot_roc_curve

In [ ]:
def build_clf(width=4, dropout_rate=0.0):
    ann = keras.models.Sequential()
    ann.add(keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    ann.add(keras.layers.Dense(units=width, activation='relu'))
    ann.add(keras.layers.Dropout(rate=dropout_rate, noise_shape=None, seed=None))
    ann.add(keras.layers.Dense(units=width, activation='relu'))
    ann.add(keras.layers.Dropout(rate=dropout_rate, noise_shape=None, seed=None))
    ann.add(keras.layers.Dense(units=1, activation='sigmoid'))
    ann.compile(optimizer='adam', loss='binary_crossentropy', 
                metrics=['accuracy', keras.metrics.AUC()])
    return ann

In [ ]:
ann = KerasClassifier(build_fn=build_clf, epochs=3)
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('ann', ann)
])

In [ ]:
params={'ann__width':[4, 6, 8],
        'ann__dropout_rate' : [0.0, 0.25],
        'ann__batch_size' : [32, ]
        }
grid_nn = GridSearchCV(estimator=pipeline, param_grid=params, cv=2, return_train_score=True, verbose=3, 
                       scoring='roc_auc')
# now fit the dataset to the GridSearchCV object. 
_ = grid_nn.fit(X_train, y_train)

In [ ]:
df = grouped_boxplot_gridsearch(grid_nn, parameter_names=['ann__width', 'ann__dropout_rate'])

## Evaluation on test data

Having optimized the hyperparameters of our chosen classifier in a cross-validation, we will use this classifier to generate our predictions. 

The most straightforward option is to use .best_estimator() to access the best performing classifier 
according to the cross-validation. Per default (as determined by the `refit` argument to the gridsearch object) the entire training data is used to fit this best estimator. 

We will use predefined functions that plot various curves for us and also report classification metrics. 

In [ ]:
# Load the test data. 
y_test = pd.read_csv('y_test_supervised.csv.zip')['class']

With `plot_conditional_distribution`, we can make conditional density plots, to visualize the degree of separation  between the positive and negative class by the classifier. 
Replace `grid_dtc` with your own trained GridsearchCV object (`grid_xx`) for evaluation.

In [ ]:
y_pred = grid_dtc.best_estimator_.predict_proba(X_test)[:, 1]
print(f'The ROC-AUC test score: {roc_auc_score(y_test, y_pred):.3f}')
_ = plot_conditional_distribution(y_test, y_pred, bw=0.01)

An even easier approach is to plot the ROC and  Precision-Recall curves using
`plot_roc_curve` and `plot_precision_recall_curve`.
Both functions require an estimator (we take our best one, stored in `.best_estimator_`), and the X_test and y_test data. See examples below. 

Replace `grid_dtc` with your own trained GridsearchCV object (`grid_xx`) for evaluation

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
_ = plot_roc_curve(estimator=grid_dtc.best_estimator_, X=X_test, y=y_test, ax=axs[0])
_ = plot_precision_recall_curve(estimator=grid_dtc.best_estimator_, X=X_test, y=y_test, ax=axs[1])

_ = axs[0].set_title('ROC Curve')
_ = axs[1].set_title('Average Precision Curve')

## Comparison of all "best" estimators

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
for clf in [grid_dtc, grid_nb, grid_lr, grid_rfc, grid_gb, grid_nn]:
    label = (type(clf.estimator).__name__ if not type(clf.estimator).__name__ == 'Pipeline' 
        else type(clf.estimator[-1]).__name__)
    _ = plot_roc_curve(estimator=clf.best_estimator_, X=X_test, y=y_test, ax=axs[0], label=label)
    _ = plot_precision_recall_curve(estimator=clf.best_estimator_, X=X_test, y=y_test, ax=axs[1], label=label)
    y_pred = clf.best_estimator_.predict_proba(X_test)[:, 1]
    print(f'{label:>30}. ROC-AUC test score: {roc_auc_score(y_test, y_pred):>6.3f},   '\
         f'Average Precision test score: {average_precision_score(y_test, y_pred):>6.3f}')
